In [1]:
# sys
import os
import csv
import sys

# math
import numpy as np
import pandas as pd
import random

# data cleaning
sys.path.append('/home/flink/workspace/yimin/Flink-SQL-Metrics-Tracer/src/cleaners')
# from seq_cleaner import sequential_cleaner
import seq_cleaner
from job_meta_data import embeddings, keys, selectivities, DAGs, DAGs_edges, src_selectivities

# # learning
import tensorflow as tf
# import tensorflow.keras as keras
# from sklearn.model_selection import train_test_split
# from keras.preprocessing.text import Tokenizer
# from keras.models import Sequential,Model
# from keras.layers import Dense,Embedding,Input,Concatenate,Flatten
import tensorflow_gnn as tfgnn

# visual
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

2023-01-24 10:17:54.542300: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-24 10:17:54.679627: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-24 10:17:54.684005: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-24 10:17:54.684022: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
from importlib import reload
reload(seq_cleaner)
sequential_cleaner = seq_cleaner.sequential_cleaner

## Raw data load, select & taste on vertex-level

In [4]:
# read files and combine
f_paths = ['/home/flink/workspace/yimin/datasets/nexmark_full_job_Nov14/test/test2.csv',
           '/home/flink/workspace/yimin/datasets/nexmark_full_job_Nov14/test/test3.csv',
           '/home/flink/workspace/yimin/datasets/nexmark_full_job_Nov14/test/test4.csv',
           '/home/flink/workspace/yimin/datasets/nexmark_full_job_Nov14/test/test5.csv',
          ]
df_raws = []
for f_path in f_paths:
    df_raws.append(pd.read_csv(f_path))
df_raw = pd.concat(df_raws, axis=0, ignore_index=True)
df_raw.shape

(6060, 2)

In [ ]:
# define wanted dict
wanted_dict = {}
wanted_dict['run_confs'] = ['qid',]
wanted_dict['vertex_level_metrics'] = ['parallelism',]
wanted_dict['vertex_id'] = 'idx'
wanted_dict['sub_level_metrics'] = ['busyTimeMsPerSecond', 
                                    'numBytesInPerSecond', 'numRecordsInPerSecond',
                                    'numBytesOutPerSecond', 'numRecordsOutPerSecond',
                                    'isBackPressured',
                                   ]

In [4]:
# do data cleaning
scleaner = sequential_cleaner(wanted_dict)
df_raw[:].apply(scleaner.clean_row, axis=1)
len(scleaner.dataset), scleaner.dataset[:1]

KeyboardInterrupt: 

In [ ]:
# make df and replace nan
df = pd.DataFrame(scleaner.dataset, columns=scleaner.get_head())
df = df.replace('NaN', np.nan)
(df.shape[0], df.shape[1], df.keys())

In [ ]:
# col name & dtype transform
df.rename(columns = {'qid':'query_idx',}, inplace = True)

df = df.astype({"query_idx": int, 'vertex_idx': int, 'parallelism':int, 'sub_idx':int,
                           'busyTimeMsPerSecond': float, 
                           'numBytesInPerSecond': float, 'numRecordsInPerSecond': float,
                           'numBytesOutPerSecond': float, 'numRecordsOutPerSecond': float,
               })

In [ ]:
df_bp = df.filter(items=['qid', 'vertex_idx', 'parallelism', 'sub_idx', 'isBackPressured'])
print('bp v-records num',df_bp[df_bp['isBackPressured'] == 'true'].shape[0])
print('nonbp v-records num',df_bp[df_bp['isBackPressured'] == 'false'].shape[0])

In [ ]:
df_bp_v = df_bp[df_bp['isBackPressured'] == 'true']
print(df_bp_v.shape)
df_bp_v0 = df_bp_v[df_bp_v['vertex_idx'] == 0]
print(df_bp_v0.shape)
print(df_bp_v0[(df_bp_v0['qid']==15)].shape, df_bp[(df_bp['qid']==15) & (df_bp['vertex_idx']==0)].shape)

In [ ]:
df_naive = df[df['busyTimeMsPerSecond'].notnull()]

df_naive['VID'] = 'q' + df_naive['query_idx'].astype(str) + 'v' + df_naive['vertex_idx'].astype(str)

print(df_naive.shape)

In [ ]:
def observe_utilization_dist(qid, vid, parallels=[2,4,6,8,12,16],):
    fig, axs = plt.subplots(1, 6, figsize=(14,2))
    ctr = 0
    for paralle in parallels:

        df2_naive = df_naive[df_naive['query_idx']==qid]
        df2_naive = df2_naive[df2_naive['vertex_idx']==vid]
        df2_naive = df2_naive[df2_naive['parallelism']==paralle]

        axs[ctr].hist(df2_naive['busyTimeMsPerSecond'], bins=50,)
        axs[ctr].set_title(" ".join(['q',str(qid),'v',str(vid),'p',str(paralle)]))
        ctr+=1
    plt.show()

In [ ]:
observe_utilization_dist(qid=20, vid=1,)

# Make TF-GNN Dataset

In [ ]:
# test of the newly-implemented cleaner class

In [ ]:
df_raw_test = df_raw[:10]

In [85]:
wanted_dict2 = {}
wanted_dict2['run_confs'] = ['qid', 'parallel', 'tps',]
wanted_dict2['job_level_metrics'] = ['pseudoSrcInputRate', 'jobIsBackPressured', 'jobVerParallelisms']
# wanted_dict2['vertex_level_metrics'] = ['parallelism',]
# wanted_dict2['vertex_id'] = 'idx'
# wanted_dict2['sub_level_metrics'] = ['busyTimeMsPerSecond', 
#                                     'numBytesInPerSecond', 'numRecordsInPerSecond',
#                                     'numBytesOutPerSecond', 'numRecordsOutPerSecond',
#                                     'isBackPressured',
#                                    ]

scleaner2 = sequential_cleaner(wanted_dict2)
df_raw_test[:].apply(scleaner2.clean_row, axis=1)
len(scleaner2.dataset), scleaner2.dataset[:10]

NameError: name 'df_raw_test' is not defined

In [ ]:
scleaner2.get_head()

### complete the dataset attributes for GNN

In [6]:
# metrics_dataset = []
# def make_job_datasets():
#     sample = ['']

In [5]:
# define wanted dict
wanted_dict = {}
wanted_dict['run_confs'] = ['qid',      # for DAG, nodes and edges
                            'parallel', # no-use
                            'tps']      # for global informations
wanted_dict['job_level_metrics'] = ['pseudoSrcInputRate',    # for global informations
                                    'sourceIsBackPressured', # for global informations
                                    'jobVerParallelisms',    # for node embeddings
                                    'jobIsBackPressured',
                                    'srcBPTime']   # y

In [6]:
# do data cleaning
scleaner_jobds = sequential_cleaner(wanted_dict)
df_raw[:].apply(scleaner_jobds.clean_row, axis=1)
len(scleaner_jobds.dataset), scleaner_jobds.dataset[:1]

(6060,
 [[4,
   2,
   10000,
   68266.66666666667,
   1677.4,
   'False',
   '[2, 2, 2, 2]',
   'False',
   0.0]])

In [7]:
# make df and replace nan
df_jobds = pd.DataFrame(scleaner_jobds.dataset, columns=scleaner_jobds.get_head())
df_jobds = df_jobds.replace('NaN', np.nan)
(df_jobds.shape[0], df_jobds.shape[1], df_jobds.keys())

(6060,
 9,
 Index(['qid', 'parallel', 'tps', 'pseudoSrcNumBytesInPerSecond',
        'pseudoSrcNumRecordsInPerSecond', 'sourceIsBackPressured',
        'jobVerParallelisms', 'jobIsBackPressured', 'srcBPTime'],
       dtype='object'))

In [8]:
df_jobds = df_jobds.astype({"qid": int, 'parallel': float, 'tps': float,
                            'pseudoSrcNumBytesInPerSecond': float, 'pseudoSrcNumRecordsInPerSecond': float,
                            'sourceIsBackPressured': str, 'jobVerParallelisms': str,
                            'jobIsBackPressured': str, 'srcBPTime':float,
               })

In [9]:
df_jobds

,qid,parallel,tps,pseudoSrcNumBytesInPerSecond,pseudoSrcNumRecordsInPerSecond,sourceIsBackPressured,jobVerParallelisms,jobIsBackPressured,srcBPTime
0,4,2.0,10000.0,6.826667e+04,1677.400000,False,"[2, 2, 2, 2]",False,0.0000
1,4,2.0,10000.0,1.354411e+05,3310.883333,False,"[2, 2, 2, 2]",False,0.0000
2,4,2.0,10000.0,2.029094e+05,4944.550000,False,"[2, 2, 2, 2]",False,0.0000
3,4,2.0,10000.0,2.717222e+05,6578.050000,False,"[2, 2, 2, 2]",False,0.0000
4,4,2.0,10000.0,3.394428e+05,8211.416667,False,"[2, 2, 2, 2]",False,0.0000
...,...,...,...,...,...,...,...,...,...
6055,20,16.0,2400000.0,6.427443e+07,287646.816667,True,"[16, 16]",True,987.3125
6056,20,16.0,2400000.0,5.596119e+07,249912.200000,True,"[16, 16]",True,995.3125
6057,20,16.0,2400000.0,4.355695e+07,194295.166667,True,"[16, 16]",True,1000.0000
6058,20,16.0,2400000.0,5.004064e+07,223510.616667,True,"[16, 16]",True,616.5000


In [10]:
class jobds_processor:
    def __init__(self, first_selectivities):
        self.DS = []
#         self.embeddings = embeddings
#         self.selectivities = selectivities
        self.first_selectivities = first_selectivities
        self.fucked_ctr = 0
    
    def get_head(self,):
        return ['qid', 'pseudoInputBytesRate', 'pseudoInputRecordsRate', 'vPs', 'tps', 'srcBP', 'isBP', 'srcBPTime', 'truePIRR']
    
    def preprocess_f(self, row):
        # res: target storing variable
        qid = row['qid'] # 1. qid
        tps = row['tps']
        input_bytes_rate =  row['pseudoSrcNumBytesInPerSecond'] # 2. (pseudo) input bytes rate
        input_records_rate = row['pseudoSrcNumRecordsInPerSecond'] # 3. (pseudo) input records rate
        sourceIsBackPressured = row['sourceIsBackPressured'] == 'True'
        srcBPTime = row['srcBPTime']
        ## ISSUE
        if sourceIsBackPressured is True:
            self.fucked_ctr += 1
            random_scalar = random.randrange(70, 130)/100
            input_bytes_rate = tps * first_selectivities[qid]['bytes'] * random_scalar
            input_records_rate = tps * first_selectivities[qid]['rcds'] * random_scalar
        ##
        v_parallels = row['jobVerParallelisms'] # 4. parallelism for each vertex
        y = row['jobIsBackPressured'] == 'True' # 5. y, whether the job is back-pressured
        true_input_records_rate = 1000/(1000-srcBPTime)*input_records_rate if srcBPTime < 1000 else input_records_rate        
        sample = [qid, input_bytes_rate, input_records_rate, v_parallels, tps,sourceIsBackPressured, y, srcBPTime, true_input_records_rate]
        self.DS.append(sample)

In [10]:
# bool('False')

In [11]:
# side data preparation
# first-selectivities
# li = [[3, 0, 0.01617300468409586, 0.6810972130718954],
#  [10, 0, 2.534979423868313e-07, 0.0010509843621399175],
#  [11, 0, 0.6726323098843693, 22.094161085754536],
#  [12, 0, 0.6713340591853196, 22.079498634422265],
#  [15, 0, 0.6932316232959345, 56.158943730848115],
#  [16, 0, 0.6731368146825398, 71.2333128161977],
#  [17, 0, 0.6564263170751338, 53.09597818988692],
#  [18, 0, 0.6703125484013426, 147.11353561752927],
#  [19, 0, 0.6271182971569771, 137.64395987670054],
#  [20, 0, 0.6591435695819582, 147.2903163747839],
#  [7, 0, 1.2670766402032947, 101.72114405502981],
#  [9, 0, 0.6792085830197133, 162.8975794798387],
#  [4, 0, 0.7134230608788854, 29.3562483091104],
#  [5, 0, 0.6746866944805664, 22.14287440593845],
#  [8, 0, 0.05863777322076979, 2.1607193068264343]]

first_selectivities = {}
for e in src_selectivities:
    first_selectivities[e[0]] = {'rcds':e[2], 'bytes':e[3]}
# first_selectivities

# non-first selectivities
# selectivities imported

In [12]:
jpc = jobds_processor(first_selectivities)

In [13]:
df_jobds.apply(jpc.preprocess_f, axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
6055    None
6056    None
6057    None
6058    None
6059    None
Length: 6060, dtype: object

In [14]:
df_jobdbs_complete = pd.DataFrame(jpc.DS, columns=jpc.get_head())
df_jobdbs_complete = df_jobdbs_complete[(df_jobdbs_complete['qid']!=10) & (df_jobdbs_complete['qid']!=15)]
df_jobdbs_complete

,qid,pseudoInputBytesRate,pseudoInputRecordsRate,vPs,tps,srcBP,isBP,srcBPTime,truePIRR
0,4,6.826667e+04,1.677400e+03,"[2, 2, 2, 2]",10000.0,False,False,0.0000,1.677400e+03
1,4,1.354411e+05,3.310883e+03,"[2, 2, 2, 2]",10000.0,False,False,0.0000,3.310883e+03
2,4,2.029094e+05,4.944550e+03,"[2, 2, 2, 2]",10000.0,False,False,0.0000,4.944550e+03
3,4,2.717222e+05,6.578050e+03,"[2, 2, 2, 2]",10000.0,False,False,0.0000,6.578050e+03
4,4,3.394428e+05,8.211417e+03,"[2, 2, 2, 2]",10000.0,False,False,0.0000,8.211417e+03
...,...,...,...,...,...,...,...,...,...
6055,20,6.118834e+08,2.732715e+06,"[16, 16]",2400000.0,True,True,987.3125,2.153864e+08
6056,20,5.835555e+08,2.606201e+06,"[16, 16]",2400000.0,True,True,995.3125,5.559895e+08
6057,20,6.118834e+08,2.732715e+06,"[16, 16]",2400000.0,True,True,1000.0000,2.732715e+06
6058,20,6.232146e+08,2.783321e+06,"[16, 16]",2400000.0,True,True,616.5000,7.257682e+06


### balance labels

In [34]:
df_jobdbs_complete[df_jobdbs_complete['isBP'] == True]

,qid,pseudoInputBytesRate,pseudoInputRecordsRate,vPs,tps,srcBP,isBP,srcBPTime,truePIRR
21,7,1.169818e+06,14539.735,"[2, 1, 2]",10000.0,True,True,0.0000,1.453973e+04
102,7,8.355840e+06,103855.250,"[2, 1, 2]",40000.0,True,True,165.0000,1.243775e+05
103,7,8.222147e+06,102193.566,"[2, 1, 2]",40000.0,True,True,633.0000,2.784566e+05
104,7,8.288993e+06,103024.408,"[2, 1, 2]",40000.0,True,True,742.0000,3.993194e+05
105,7,4.679270e+06,58158.940,"[2, 1, 2]",40000.0,True,True,763.0000,2.453964e+05
...,...,...,...,...,...,...,...,...,...
6055,20,5.835555e+08,2606200.760,"[16, 16]",2400000.0,True,True,987.3125,2.054148e+08
6056,20,6.742049e+08,3011047.480,"[16, 16]",2400000.0,True,True,995.3125,6.423568e+08
6057,20,6.912016e+08,3086956.240,"[16, 16]",2400000.0,True,True,1000.0000,3.086956e+06
6058,20,6.005522e+08,2682109.520,"[16, 16]",2400000.0,True,True,616.5000,6.993767e+06


In [17]:
pos_set = df_jobdbs_complete[df_jobdbs_complete['isBP'] == True]
neg_set = df_jobdbs_complete[df_jobdbs_complete['isBP'] == False]
pos_set.shape[0], neg_set.shape[0]

(1990, 3470)

In [18]:
pos_size = pos_set.shape[0]
sampled_neg_set = neg_set.sample(pos_size)
df_jobdbs_complete = pd.concat([pos_set, sampled_neg_set], ignore_index=True)
df_jobdbs_complete

,qid,pseudoInputBytesRate,pseudoInputRecordsRate,vPs,tps,srcBP,isBP,srcBPTime,truePIRR
0,7,1.621033e+06,2.014792e+04,"[2, 1, 2]",10000.0,True,True,0.0,2.014792e+04
1,7,7.821066e+06,9.720851e+04,"[2, 1, 2]",40000.0,True,True,165.0,1.164174e+05
2,7,6.885212e+06,8.557673e+04,"[2, 1, 2]",40000.0,True,True,633.0,2.331791e+05
3,7,7.085752e+06,8.806925e+04,"[2, 1, 2]",40000.0,True,True,742.0,3.413537e+05
4,7,6.550979e+06,8.142252e+04,"[2, 1, 2]",40000.0,True,True,763.0,3.435549e+05
...,...,...,...,...,...,...,...,...,...
3975,17,1.693013e+05,2.122200e+03,"[3, 3]",15000.0,False,False,0.0,2.122200e+03
3976,5,6.553600e+04,2.029417e+03,"[3, 3, 3, 3]",15000.0,False,False,0.0,2.029417e+03
3977,8,2.878123e+05,7.510750e+03,"[3, 3, 3, 3]",300000.0,False,False,0.0,7.510750e+03
3978,8,8.519680e+04,2.309983e+03,"[3, 3, 3, 3]",60000.0,False,False,0.0,2.309983e+03


In [19]:
# test filed
1677.400000/6.826667e+04*1e+06

24571.287862730085

In [20]:
df_jobdbs_complete[(df_jobdbs_complete['qid']==4) & (df_jobdbs_complete['vPs']=='[2, 2, 2, 2]')].sort_values(by=['pseudoInputBytesRate'])[:]

,qid,pseudoInputBytesRate,pseudoInputRecordsRate,vPs,tps,srcBP,isBP,srcBPTime,truePIRR
2137,4,6.826667e+04,1677.400000,"[2, 2, 2, 2]",10000.0,False,False,0.0,1677.400000
3692,4,2.029094e+05,4944.550000,"[2, 2, 2, 2]",10000.0,False,False,0.0,4944.550000
2342,4,2.717222e+05,6578.050000,"[2, 2, 2, 2]",10000.0,False,False,0.0,6578.050000
3087,4,3.394428e+05,8211.416667,"[2, 2, 2, 2]",10000.0,False,False,0.0,8211.416667
2508,4,3.883008e+05,9411.650000,"[2, 2, 2, 2]",20000.0,False,False,0.0,9411.650000
3162,4,4.060710e+05,9801.150000,"[2, 2, 2, 2]",10000.0,False,False,0.0,9801.150000
2414,4,4.063232e+05,9800.833333,"[2, 2, 2, 2]",10000.0,False,False,0.0,9800.833333
2826,4,4.071633e+05,9801.000000,"[2, 2, 2, 2]",10000.0,False,False,0.0,9801.000000
2616,4,4.079616e+05,9800.966667,"[2, 2, 2, 2]",10000.0,False,False,0.0,9800.966667
3580,4,5.958315e+05,14407.366667,"[2, 2, 2, 2]",100000.0,False,False,0.0,14407.366667


### Define functions and construct dataset file in tf-gnn's GraphTensor

In [35]:
def make_graph_tensor(**kwargs):
    # function for job-level 
    
    nodes = kwargs['nodes']     # {source: [parallelisms], 
                                #  operate: [parallelisms, 
#                                            bytes_selectivities,
#                                            records_selectivities,
#                                            utilization_embeddings]}
    edges = kwargs['edges']     # {[[sources,],[targets,]]}, {src2op, op2op}
    context = kwargs['context'] # [3], [bytes_inpute_rate, records_input_rate, label_isBP,]
    
    graph = tfgnn.GraphTensor.from_pieces(
        # make node_sets
        node_sets={
            'source': tfgnn.NodeSet.from_fields(
                sizes=tf.constant([len(nodes['source'])]),
                features={
                    'parallelism': tf.constant(nodes['source'])
                }),
            'operate': tfgnn.NodeSet.from_fields(
                sizes=tf.constant([len(nodes['operate'][0])]),
                features={
                    'parallelism': tf.constant(nodes['operate'][0]),
                    'bytes_selectivity': tf.constant(nodes['operate'][1]),
                    'records_selectivity': tf.constant(nodes['operate'][2]),
                    'utilization_embedding': tf.constant(nodes['operate'][3]),
                }),
        },
        # make edge_sets
        edge_sets={
            'src2op': tfgnn.EdgeSet.from_fields(
                sizes=tf.constant([len(edges['src2op'][0])]),
                adjacency=tfgnn.Adjacency.from_indices(
                    source=('source', tf.constant(edges['src2op'][0])),
                    target=('operate', tf.constant(edges['src2op'][1])))),
            'op2op': tfgnn.EdgeSet.from_fields(
                sizes=tf.constant([len(edges['op2op'][0])]),
                adjacency=tfgnn.Adjacency.from_indices(
                    source=('operate', tf.constant(edges['op2op'][0], dtype=tf.int64)),
                    target=('operate', tf.constant(edges['op2op'][1], dtype=tf.int64)))),
                
        },
        # make context
        context=tfgnn.Context.from_fields(
                features={
                    'bytes_input_rate': tf.constant([context[0],]),
                    'records_input_rate': tf.constant([context[1],]),
                    'label_isBP': tf.constant([context[2]]),
                })
    )

    return graph

In [36]:
class graph_tensor_dataset_maker:
    def __init__(self, embeddings, keys, DAG, selectivities, o_fp='./tfgnn_test/dummy/try_write.tfrecord', debug=False):
        self.embeddings = embeddings
        self.keys = list(keys) # ndarray -> list
        self.DAG = DAG
        self.selectivities = selectivities
        self.o_fp = o_fp
        self.debug = debug
        self.dataset_buffer = []
    
    def buffer_output(self, demo=False):
        with tf.io.TFRecordWriter(self.o_fp) as writer:
            for graph in self.dataset_buffer:
                example = tfgnn.write_example(graph)
                if demo:
                    print(example)
                writer.write(example.SerializeToString())

    def make(self, row):
        # encode sample row data
        qid = row['qid']
        inBrate = row['pseudoInputBytesRate']
        inRrate = row['pseudoInputRecordsRate']
        parallels = eval(row['vPs'])
        parallels = [float(e) for e in parallels]
        label = row['isBP']
        vnum = len(parallels)
        
        # find coress-information        
        dag = self.DAG[qid]
        embeds = [] # length should be vnum-1, since no embedding for the source vertex
        for i in range(1, vnum):
            vkeyname = ''.join(['q', str(qid), 'v', str(i)])
            embeds.append(self.embeddings[self.keys.index(vkeyname)])
        
        # make context
        context = [row['pseudoInputBytesRate'],
                   row['pseudoInputRecordsRate'],
                   1 if row['isBP'] else 0]
        
        # make edges
        src2op_edges = []
        op2op_edges = []
        for i in range(len(dag[0])):
            s = dag[0][i]
            t = dag[1][i]
            if s == 0:
                src2op_edges.append([s,t])
            else:
                op2op_edges.append([s,t])
        edges = {
            'src2op': [[e[0] for e in src2op_edges],
                       [e[1]-1 for e in src2op_edges]],
            'op2op': [[e[0]-1 for e in op2op_edges],
                      [e[1]-1 for e in op2op_edges]],
        }
        
        # make nodes
        nodes = {}
        nodes['source'] = [parallels[0]]
        vselects = []
        for i in range(1, vnum):
            vselects.append([
                self.selectivities['true_selectivity_bytes'][(qid, i)],
                self.selectivities['true_selectivity_records'][(qid, i)],
            ])
        nodes['operate'] = [
            parallels[1:],
            [e[0] for e in vselects],
            [e[1] for e in vselects],
            embeds,
        ]
        
        if self.debug:
            print('nodes', nodes)
            print('edges', edges)
            print('context', context)
        
        # convert to tfgnn graph tensor
        graph = make_graph_tensor(nodes=nodes, edges=edges, context=context)
        
        # write it down
#         self.single_writer(graph, demo=self.debug)
#         print('.', end='')
        self.dataset_buffer.append(graph)
#         return graph

In [19]:
# dataset_opath = '/home/flink/workspace/yimin/datasets/nexmark_full_job_Nov14/test/tfgnn_dataset/nexmark_isBP.tfrecord'
dataset_opath = '/home/flink/workspace/yimin/datasets/nexmark_full_job_Nov14/test/tfgnn_dataset/nexmark_isBP_fixed.tfrecord'

In [20]:
amaker = graph_tensor_dataset_maker(embeddings, keys, DAGs, selectivities, o_fp=dataset_opath, debug=False)

In [21]:
df_jobdbs_complete[:].apply(amaker.make, axis=1)

2023-01-11 06:32:00.602977: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-11 06:32:00.603045: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-11 06:32:00.603090: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (camel-sane): /proc/driver/nvidia/version does not exist
2023-01-11 06:32:00.610705: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


KeyboardInterrupt: 

In [ ]:
amaker.buffer_output()

In [ ]:
plt.hist(df_jobdbs_complete['pseudoInputBytesRate'], bins=150)
plt.show()
plt.hist(df_jobdbs_complete['pseudoInputRecordsRate'], bins=150)
plt.show()

In [ ]:
df_jobdbs_complete[df_jobdbs_complete['pseudoInputBytesRate'] == 0]

In [ ]:
# df_jobdbs_complete[df_jobdbs_complete['pseudoInputRecordsRate'] == 0]
df_jobdbs_complete

In [ ]:
df_jobdbs_complete.drop_duplicates()

### split the whole data set to train set, val set and test set

In [54]:
# record_fp = './tfgnn_dataset/nexmark_isBP.tfrecord'
record_fp = './tfgnn_dataset/nexmark_isBP_fixed.tfrecord'
schema_fp = './tfgnn_dataset/schema_poc.pbtxt'

In [55]:
dataset = tf.data.TFRecordDataset(filenames=[record_fp])

In [56]:
dataset = dataset.shuffle(buffer_size=10000, seed=8888)

In [57]:
dataset_length = 0
for v in list(dataset.as_numpy_iterator()):
    dataset_length += 1
dataset_length

3980

In [58]:
train_len = int(0.7*dataset_length)
val_len = int(0.15*dataset_length)
test_len = dataset_length - train_len - val_len
train_fp = './tfgnn_dataset/nexmark_isBP_train_fixed.tfrecord'
val_fp = './tfgnn_dataset/nexmark_isBP_val_fixed.tfrecord'
test_fp = './tfgnn_dataset/nexmark_isBP_test_fixed.tfrecord'

In [59]:
def write_back(buffer, fp):
    # log
    print('writing to file', fp)
    # write
    with tf.io.TFRecordWriter(fp) as writer:
        for sample in buffer:
            writer.write(sample)
    # cleans the buffer
    buffer = []

In [60]:
buffer = []
itr = 0
for v in list(dataset.as_numpy_iterator()):
    buffer.append(v)
    if itr == train_len-1:
        write_back(buffer, train_fp)
        buffer = []
    elif itr == train_len+val_len-1:
        write_back(buffer, val_fp)
        buffer = []
    elif itr == train_len+val_len+test_len-1:
        write_back(buffer, test_fp)
        buffer = []
    itr += 1

writing to file ./tfgnn_dataset/nexmark_isBP_train_fixed.tfrecord
writing to file ./tfgnn_dataset/nexmark_isBP_val_fixed.tfrecord
writing to file ./tfgnn_dataset/nexmark_isBP_test_fixed.tfrecord


In [37]:
dataset_length

3980

In [38]:
test_len

597

In [86]:
val_len

597

In [87]:
train_len

2786

### Experiments: analyse the naive resource recommender

In [77]:
# visual test for model serving
from utils import gtensor2serial, make_graph_tensor, graph_tensor_dataset_maker, make_vPs, rcmmdr

In [78]:
import utils
reload(utils)
rcmmdr = utils.rcmmdr

In [89]:
# model_path = './tfgnn_model/exported_keras_model_12_30/'
# model_path = './tfgnn_model/exported_keras_model_1_8/'
# model_path = './tfgnn_model/exported_keras_model_1_10/' # -30
# model_path = './tfgnn_model/exported_keras_model_1_11/' # -30
model_path = './tfgnn_model/exported_keras_model_1_24/' # -350
e_whole_model = tf.saved_model.load(model_path)
amaker = graph_tensor_dataset_maker(embeddings, keys, DAGs, selectivities, o_fp='', debug=False)

In [105]:
# example_df = df_jobdbs_complete[(df_jobdbs_complete['qid']==5) & (df_jobdbs_complete['vPs']=='[2, 2, 2, 2]')].sort_values(by=['pseudoInputBytesRate'])[:]
# example_df = df_jobdbs_complete[(df_jobdbs_complete['qid']==5) & (df_jobdbs_complete['vPs']=='[3, 3, 3, 3]')].sort_values(by=['pseudoInputBytesRate'])[:]
example_df = df_jobdbs_complete[(df_jobdbs_complete['qid']==5) & (df_jobdbs_complete['vPs']=='[4, 4, 4, 4]')].sort_values(by=['pseudoInputBytesRate'])[:]
# example_df = df_jobdbs_complete[(df_jobdbs_complete['qid']==16) & (df_jobdbs_complete['vPs']=='[6, 6]')].sort_values(by=['pseudoInputBytesRate'])[:]
# example_df = df_jobdbs_complete[(df_jobdbs_complete['qid']==16) & (df_jobdbs_complete['vPs']=='[8, 8]')].sort_values(by=['pseudoInputRecordsRate'])[:]
# example_df = df_jobdbs_complete[(df_jobdbs_complete['qid']==11) & (df_jobdbs_complete['vPs']=='[12, 12]')].sort_values(by=['pseudoInputRecordsRate'])[:]
# example_df = df_jobdbs_complete[(df_jobdbs_complete['qid']==5) & (df_jobdbs_complete['vPs']=='[16, 16, 16, 16]')].sort_values(by=['pseudoInputRecordsRate'])[:]
# e_whole_model.signatures['serving_default'](example_df)['logits']
example_serias = example_df.apply(amaker.make, axis=1).apply(gtensor2serial)
example_inputtensor = tf.constant(example_serias, dtype=tf.string)
example_df['predicted'] = e_whole_model.signatures['serving_default'](example_inputtensor)['logits'] <= -4.948218e+05
example_df

,qid,pseudoInputBytesRate,pseudoInputRecordsRate,vPs,tps,srcBP,isBP,srcBPTime,truePIRR,predicted
490,5,8.410453e+04,2692.750000,"[4, 4, 4, 4]",20000.0,False,False,0.00,2.692750e+03,False
530,5,1.770863e+05,5482.266667,"[4, 4, 4, 4]",40000.0,False,False,0.00,5.482267e+03,False
491,5,1.862315e+05,5759.900000,"[4, 4, 4, 4]",20000.0,False,False,0.00,5.759900e+03,False
492,5,2.883584e+05,8827.450000,"[4, 4, 4, 4]",20000.0,False,False,0.00,8.827450e+03,False
570,5,3.588096e+05,10994.500000,"[4, 4, 4, 4]",80000.0,False,False,0.00,1.099450e+04,False
531,5,3.797018e+05,11616.900000,"[4, 4, 4, 4]",40000.0,False,False,0.00,1.161690e+04,False
493,5,3.888469e+05,11894.733333,"[4, 4, 4, 4]",20000.0,False,False,0.00,1.189473e+04,False
494,5,4.909739e+05,14961.816667,"[4, 4, 4, 4]",20000.0,False,False,0.00,1.496182e+04,False
532,5,5.812250e+05,17751.200000,"[4, 4, 4, 4]",40000.0,False,False,0.00,1.775120e+04,False
495,5,5.925547e+05,18028.883333,"[4, 4, 4, 4]",20000.0,False,False,0.00,1.802888e+04,False


In [98]:
print(
# rcmmdr(4, 6.532301e+06, 1.571662e+05, 4, amaker, e_whole_model, thres=2500), # f-f
# rcmmdr(4, 1.416626e+07, 3.323667e+05, 4, amaker, e_whole_model, thres=2500), # t-f
# rcmmdr(4, 1.517814e+07, 3.561072e+05, 4, amaker, e_whole_model, thres=2500), # t-t
# rcmmdr(16, 1.170801e+07, 110411.8, 2, amaker, e_whole_model, thres=2500,candidates=[4,6,8,12,16]), # f-f
rcmmdr(12, 2.266453e+05, 8.158983e+03, 2, amaker, e_whole_model, thres=-350, candidates=[4,6,8,12,16])
)

{'qid': 12, 'pseudoInputBytesRate': 226645.3, 'pseudoInputRecordsRate': 8158.983, 'vPs': '[4, 4,]', 'isBP': 0}


In [41]:
# for demos for query 12

# 2-vertex job
#3 (fails)
# parallels=[4, 6, 8, 12, 16]
# gt_input_bytes=[7.804245e+06, 1.170801e+07, 1.560958e+07, 2.341929e+07, 3.123500e+07,]
# gt_input_records=[73607.2166, 1.104118e+05, 1.472151e+05, 2.208283e+05, 2.944836e+05,]
#11
# 9.570750e+04	2942.383333	
# parallels=[2, 3, 4, 6, 8, 12, 16]
# gt_input_bytes=[3.036870e+06, 4.574413e+06, 6.099217e+06, 9.140087e+06, 1.218423e+07, 1.181505e+07, 9.738103e+06]
# gt_input_records=[92012.6666, 138612.85000, 184756.28333, 2.768553e+05, 3.690340e+05, 3.590406e+05, 2.944355e+05]
# gt_input_bytes_r=[4.8695e+06, 4.949590e+06, 7.624611e+06, 9.226420e+06, 1.819655e+07, 1.691510e+07, 1.819655e+07]
# 12
# 1.092267e+05	3346.600000	
# parallels=[2, 3, 4, 6, 8, 12, 16]
# gt_input_bytes=[9.647991e+06, 9.184324e+06, 1.221660e+07, 1.822119e+07, 2.429529e+07, 3.682904e+07, 5.046108e+07, ]
# gt_input_records=[292354.800, 278330.78333, 370125.20000, 5.520395e+05, 7.360651e+05, 1.115595e+06, 1.528353e+06, ]
# gt_input_bytes_r=[1.1785e+07, 1.054790e+07, 1.485619e+07, 2.198716e+07, 3.407020e+07, 4.397433e+07, 5.282202e+07, ]
#16
# 3.025579e+05	2867.050000	
# parallels=[2, 3, 4, 6, 8, 12, 16]
gt_input_bytes=[3.907038e+06, 5.852911e+06, 7.804245e+06, 1.170801e+07, 1.560958e+07, 2.341929e+07, 3.123500e+07]
gt_input_records=[36855.2166, 55204.433333, 73607.216667, 1.104118e+05, 0, 2.208283e+05, 2.944836e+05]
gt_input_bytes_r=[6.9640e+06, 1.044611e+07, 1.373198e+07, 2.442330e+07, 2.981801e+07, 4.178444e+07, 6.591349e+07]
#17
# 2.326528e+05	2886.500000
# parallels=[2, 3, 4, 6, 8, 12, 16]
# gt_input_bytes=[2.288408e+07, 2.240730e+07, 2.981209e+07, 4.472450e+07, 5.962601e+07, 9.187437e+07, 5.963472e+07]
# gt_input_bytes_r=[2.3300e+07, 2.567858e+07, 4.184657e+07, 5.135716e+07, 7.323150e+07, 9.986114e+07, 9.637392e+07]
#18
# 6.477141e+05	2954.233333
# parallels=[2, 3, 4, 6, 8, 12, 16]
# gt_input_bytes=[2.020803e+07, 3.031695e+07, 4.042588e+07, 6.063281e+07, 8.083702e+07, 9.928267e+07, 6.468185e+07]
# gt_input_bytes_r=[3.0084e+07, 4.268687e+07, 5.935508e+07, 8.659337e+07, 1.138317e+08, 1.097662e+08, 1.138317e+08]
#19
# 6.411605e+05	2921.083333
# parallels=[2, 3, 4, 6, 8, 12, 16]
# gt_input_bytes=[2.644596e+07, 3.031153e+07, 4.041714e+07, 6.062298e+07, 8.083377e+07, 4.851357e+07, 7.669787e+07]
# gt_input_bytes_r=[3.4254e+07, 5.717082e+07, 6.947340e+07, 1.056575e+08, 8.877157e+07, 1.042101e+08, 9.649084e+07]
#20
# 6.582835e+05	2946.433333	
# parallels=[2, 3, 4, 6, 8, 12, 16]
# gt_input_bytes=[8.396733e+06, 1.252284e+07, 1.672861e+07, 2.504895e+07, 3.339769e+07, 4.080353e+07, 3.356863e+07]
# gt_input_bytes_r=[1.9357e+07, 1.288921e+07, 3.304926e+07, 5.665587e+07, 3.625976e+07, 6.175490e+07, 5.514505e+07]

# 3-vertex job
#9
# 7.411029e+05	3094.316667
# parallels=[2, 3, 4, 6, 8, 12, 16]
# gt_input_bytes=[9.460048e+06, 1.425244e+07, 1.193028e+07, 1.800329e+07, 3.033771e+07, 2.828261e+07, 3.797647e+07]
# gt_input_bytes_r=[1.7976e+07, 1.428370e+07, 1.360353e+07, 2.069679e+07, 3.109377e+07, 4.780668e+07, 5.596879e+07]

# 4-vertex job
#4
# 1.354411e+05	3310.883333
# parallels=[2, 3, 4, 6, 8, 12, 16]
# gt_input_bytes=[1.936043e+06, 2.442308e+06, 3.277346e+06, 4.905370e+06, 6.532301e+06, 9.889929e+06, 1.317929e+07]
# gt_input_bytes_r=[3.2464e+06, 4.616684e+06, 5.902609e+06, 8.980398e+06, 1.264845e+07, 1.254726e+07, 2.529690e+07]
#5
# 9.502720e+04	2889.000000
# parallels=[2, 3, 4, 6, 8, 12, 16]
# gt_input_bytes=[1.214601e+06, 1.823539e+06, 2.431386e+06, 3.647971e+06, 4.864410e+06, 7.301064e+06, 9.735919e+06]
# gt_input_bytes_r=[2.4506e+06, 3.536323e+06, 4.528975e+06, 6.886523e+06, 5.707749e+06, 1.563427e+07, 1.761957e+07]


In [42]:
tmp = 3.025579e+05
tmp2 = 2867.050000
demo_res = []
demo_res_x = []
for i in np.arange(0,1,1):
    br = tmp * i
    rr = tmp2 * i
    demo_res.append(eval(rcmmdr(qid=16, bRate=br, rRate=rr, vnum=2, amaker=amaker, e_whole_model=e_whole_model, thres=-350)['vPs'])[0])
    demo_res_x.append(br)
plt.plot(demo_res_x, demo_res, label='recommend')
plt.plot(gt_input_bytes, parallels, label='GT\'s lower bound', alpha=0.5)
plt.plot(gt_input_bytes_r, parallels, label='GT\'s upper bound', alpha=0.5)
plt.fill_betweenx(parallels, gt_input_bytes, gt_input_bytes_r, facecolor='gray', alpha=0.2)
plt.grid()
plt.legend()
plt.show()

fucked


TypeError: tuple indices must be integers or slices, not str

In [73]:
def make_vPs(vnum, propose_parallel, source_parallel):
        if source_parallel:
            return '[' + str(source_parallel)+',' + ' '.join([str(propose_parallel)+',' for i in range(vnum-1)]) + ']'
        return '[' + ' '.join([str(propose_parallel)+',' for i in range(vnum)]) + ']'

def rcmmdr(qid, bRate, rRate,  vnum, source_parallel=None, thres=105):
    graph_row = {
        'qid': qid,
        'pseudoInputBytesRate': bRate,
        'pseudoInputRecordsRate': rRate,
        'vPs': make_vPs(vnum, 2, source_parallel),
        'isBP': 0
    }
    
    fuck = []
    ress = []
    
#     for i in range(50):
    for propose_parallel in [3,4,6,8,12,16,32,64]:
#         print(graph_row)
        serialized = gtensor2serial(amaker.make(graph_row))
        serialized_ts = tf.constant([serialized,], dtype=tf.string)
        res = e_whole_model.signatures['serving_default'](serialized_ts)['logits']
        ress.append(float(res))
        if res < thres:
            return graph_row
#             return ress
            fuck.append(0)
        else:
            fuck.append(1)
#         propose_parallel += 1
        graph_row['vPs'] = make_vPs(vnum, propose_parallel, source_parallel)
    return ress

In [38]:
int('2')

2

In [114]:
import random
for i in range(20):
    print (random.randrange(70, 130)/100)

0.82
0.86
1.22
0.93
0.72
1.18
1.19
0.83
1.12
1.06
0.72
1.06
0.89
1.05
0.72
0.86
1.06
0.93
0.78
1.18


In [14]:
input()

Contract 1 gives the process of assignment to different attributes defined in attribute allocation  contract. Because only the CA has the right to initialize the industrial metaverse, the contract will  verifies the sender first. In the blockchain, the attribute is stored in the form of key–value pairs.  For instance, the key is  , with two values that are authoritylist and threshold. The former one  indicates the affiliation AA of  and the later one represents the minimum value to recover the  public key of  .


'Contract 1 gives the process of assignment to different attributes defined in attribute allocation  contract. Because only the CA has the right to initialize the industrial metaverse, the contract will  verifies the sender first. In the blockchain, the attribute is stored in the form of key–value pairs.  For instance, the key is  , with two values that are authoritylist and threshold. The former one  indicates the affiliation AA of  and the later one represents the minimum value to recover the  public key of  .'